In this notebook, we will be using a pre trained word emedding layer provided by Google on tensorflow Hub. This will be followed by 2 dense layers

In [1]:
import numpy as np

import tensorflow as tf

!pip install tensorflow-hub

import tensorflow_hub as hub

Let us import our dataset and remove the one NULL data point.

In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split
df = pd.read_csv('drive/My Drive/dataset_final.csv', engine='python')
df.drop(['Unnamed: 0'], axis=1, inplace=True)
sr = pd.isnull(df['title'])
sr.loc[sr==True]
df.drop(83369, axis=0, inplace=True)
df = df.reset_index(drop=True)
sr = pd.isnull(df['full_text'])
drop_arr = sr.loc[sr==True].index.tolist()
df.drop(drop_arr, axis=0, inplace=True)
df = df.reset_index(drop=True)

We will extract our full text and flair columns and split the data into train and test sets

In [0]:
target_flairs = df['flair'].index.tolist()
X_text = list(df['full_text'])
Y = list(df['flair'])
x_train, x_test, y_train, y_test = train_test_split(X_text, Y, test_size=0.3)

We must convert the format of our output into one hot vectors that tensorflow can understand

In [0]:
from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
encoder.fit(y_train)
y_train_bin = encoder.transform(y_train)
y_test_bin = encoder.transform(y_test)

We can utilize class weights to perform oversampling directly in tensorflow. The model in this notebook does not use it eventually as it ended up giving worse results than the model without using any class weights

In [0]:
from sklearn.utils.class_weight import compute_class_weight
class_weights = compute_class_weight('balanced' ,encoder.classes_, y_train)

We will now build and compile the model

In [0]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim-with-oov/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[],
                           dtype=tf.string, trainable=True)

In [0]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(10))
model.add(tf.keras.layers.Activation('softmax'))

In [0]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [0]:
x_train_tensor = tf.convert_to_tensor(x_train)

In [0]:
class_weights = {i:class_weights[i] for i in range(len(class_weights))}

In [24]:
model.fit(x=x_train_tensor,y=y_train_bin, batch_size=128, epochs=10, verbose=1)

Epoch 1/10
580/580 [==============================] - 3s 5ms/step - loss: 1.0912 - accuracy: 0.6362
Epoch 2/10
580/580 [==============================] - 3s 5ms/step - loss: 0.8877 - accuracy: 0.6984
Epoch 3/10
580/580 [==============================] - 3s 5ms/step - loss: 0.8408 - accuracy: 0.7127
Epoch 4/10
580/580 [==============================] - 3s 5ms/step - loss: 0.8068 - accuracy: 0.7241
Epoch 5/10
580/580 [==============================] - 3s 5ms/step - loss: 0.7805 - accuracy: 0.7323
Epoch 6/10
580/580 [==============================] - 3s 5ms/step - loss: 0.7557 - accuracy: 0.7392
Epoch 7/10
580/580 [==============================] - 3s 5ms/step - loss: 0.7305 - accuracy: 0.7503
Epoch 8/10
580/580 [==============================] - 3s 5ms/step - loss: 0.7090 - accuracy: 0.7574
Epoch 9/10
580/580 [==============================] - 3s 5ms/step - loss: 0.6877 - accuracy: 0.7643
Epoch 10/10
580/580 [==============================] - 3s 5ms/step - loss: 0.6671 - accuracy: 0.7713

In [0]:
x_test_tensor = tf.convert_to_tensor(x_test)

In [17]:
model.evaluate(x_test_tensor, y_test_bin)

994/994 [==============================] - 3s 3ms/step - loss: 1.2526 - accuracy: 0.5853


[1.2525954246520996, 0.5852799415588379]

We see that this result is better than the one we got from using the RNN from 